# Import dependencies

In [1]:
import os
import sys

sys.path.insert(0, os.path.dirname(os.getcwd())) 

In [2]:
import time
import gc
import json

import numpy as np
import pandas as pd

from transformers import BertTokenizer, BertModel
from transformers import logging

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

from InputDataset import InputDataset

import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence

from torch import cuda

from models.BERT_Dropout_CNN_BiLSTM_Linear import BERT_Dropout_CNN_BiLSTM_Linear

In [3]:
device = 'cuda' if cuda.is_available() else 'cpu'
logging.set_verbosity_error() 

In [4]:
print(torch.cuda.get_device_name(0))
print(f"Memory: {torch.cuda.get_device_properties(0).total_memory // 1024 ** 3} GB")

NVIDIA GeForce RTX 2060 SUPER
Memory: 8 GB


In [5]:
def clear_memory():
    torch.cuda.empty_cache()

    with torch.no_grad():
        torch.cuda.empty_cache()

    gc.collect()

# Load Data

In [6]:
DATASET = 'ATE_SemEval16_Restaurants_train.json'

In [7]:
df = pd.json_normalize(json.load(open(DATASET)))

In [8]:
df.head()

,text,tokens,iob_aspect_tags
0,Judging from previous posts this used to be a ...,"[Judging, from, previous, posts, this, used, t...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
1,"We, there were four of us, arrived at noon - t...","[We, ,, there, were, four, of, us, ,, arrived,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"They never brought us complimentary noodles, i...","[They, never, brought, us, complimentary, nood...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,The food was lousy - too sweet or too salty an...,"[The, food, was, lousy, -, too, sweet, or, too...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"
4,The food was lousy - too sweet or too salty an...,"[The, food, was, lousy, -, too, sweet, or, too...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"


# Train & Validate

In [9]:
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4

EPOCHS = 2

LEARNING_RATE = 1e-5

TRAIN_SPLIT = 0.8

NO_RUNS = 10

In [10]:
SEQ_LEN = 512

In [11]:
MODEL_OUTPUT = '../../../results/ATE/SemEval16 - Task 5 - Restaurants/models/bert_pre_trained_dropout_cnn_bilstm_linear.pth'
STATS_OUTPUT = '../../../results/ATE/SemEval16 - Task 5 - Restaurants/stats/bert_pre_trained_dropout_cnn_bilstm_linear.csv'

In [12]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [13]:
best_accuracy = 0.0

In [14]:
def create_mini_batch(samples):
    ids_tensors = [s[1] for s in samples]
    ids_tensors[0] = torch.nn.ConstantPad1d((0, SEQ_LEN - len(ids_tensors[0])), 0)(ids_tensors[0])
    ids_tensors = pad_sequence(ids_tensors, batch_first=True).to(device)

    tags_tensors = [s[2] for s in samples]
    tags_tensors[0] = torch.nn.ConstantPad1d((0, SEQ_LEN - len(tags_tensors[0])), 0)(tags_tensors[0])
    tags_tensors = pad_sequence(tags_tensors, batch_first=True).to(device)
    
    masks_tensors = torch.zeros(ids_tensors.shape, dtype=torch.long).to(device)
    masks_tensors = masks_tensors.masked_fill(ids_tensors != 0, 1).to(device)
    
    return ids_tensors, tags_tensors, masks_tensors

In [15]:
def train(epoch, model, loss_fn, optimizer, dataloader):
    model.train()

    dataloader_len = len(dataloader)

    for _,data in enumerate(dataloader, 0):
        optimizer.zero_grad()

        ids_tensors, tags_tensors, masks_tensors = data

        outputs = model(ids_tensors, masks_tensors)

        loss = loss_fn(outputs.view(-1, 3), tags_tensors.view(-1))
        
        if _ % (dataloader_len // 10) == 0:
            print(f"Epoch: {epoch}/{EPOCHS}, Batch: {_}/{dataloader_len}, Loss: {loss.item()}")
        
        loss.backward()
        
        optimizer.step()

In [16]:
def validation(model, dataloader):
    model.eval()
    
    fin_targets=[]
    fin_outputs=[]

    with torch.no_grad():
        for _, data in enumerate(dataloader, 0):
            ids_tensors, tags_tensors, masks_tensors = data
            ids_tensors = ids_tensors.to(device)
            tags_tensors = tags_tensors.to(device)
            masks_tensors = masks_tensors.to(device)

            outputs = model(ids_tensors, masks_tensors)
            
            _, predictions = torch.max(outputs, dim=2)

            fin_outputs += list([int(p) for pred in predictions for p in pred])
            fin_targets += list([int(tag) for tags_tensor in tags_tensors for tag in tags_tensor])

    return fin_outputs, fin_targets

In [17]:
results = pd.DataFrame(columns=['accuracy','precision_score_micro','precision_score_macro','recall_score_micro','recall_score_macro','f1_score_micro','f1_score_macro', 'execution_time'])

In [18]:
for i in range(NO_RUNS):
    # clear cache cuda
    torch.cuda.empty_cache()
    with torch.no_grad():
        torch.cuda.empty_cache()
    gc.collect()

    start_time = time.time()

    print(f"Run {i + 1}/{NO_RUNS}")

    train_dataset = df.sample(frac=TRAIN_SPLIT)
    test_dataset = df.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    training_set = InputDataset(train_dataset, tokenizer)
    testing_set = InputDataset(test_dataset, tokenizer)

    train_dataloader = DataLoader(
        training_set,
        sampler = RandomSampler(train_dataset),
        batch_size = TRAIN_BATCH_SIZE,
        drop_last = True,
        collate_fn=create_mini_batch
    )

    validation_dataloader = DataLoader(
        testing_set,
        sampler = SequentialSampler(testing_set),
        batch_size = VALID_BATCH_SIZE,
        drop_last = True,
        collate_fn=create_mini_batch
    )

    model = BERT_Dropout_CNN_BiLSTM_Linear(BertModel.from_pretrained('bert-base-uncased'), bert_seq_len=SEQ_LEN, dropout=0.3, bilstm_in_features=256, conv_out_channels=SEQ_LEN, conv_kernel_size=3, no_out_labels=3, device=device).to(device)

    optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE)
    loss_fn = torch.nn.CrossEntropyLoss()

    for epoch in range(EPOCHS):
        train(epoch, model, loss_fn, optimizer, train_dataloader)

    outputs, targets = validation(model, validation_dataloader)
    
    accuracy = accuracy_score(targets, outputs)
    precision_score_micro = precision_score(targets, outputs, average='micro')
    precision_score_macro = precision_score(targets, outputs, average='macro')
    recall_score_micro = recall_score(targets, outputs, average='micro')
    recall_score_macro = recall_score(targets, outputs, average='macro')
    f1_score_micro = f1_score(targets, outputs, average='micro')
    f1_score_macro = f1_score(targets, outputs, average='macro')

    execution_time = time.time() - start_time

    results.loc[i] = [accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro, execution_time]

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model.bert, MODEL_OUTPUT)

    del train_dataset
    del test_dataset
    del training_set
    del testing_set
    del model
    del loss_fn
    del optimizer
    del outputs
    del targets

Run 1/10
Epoch: 0/2, Batch: 0/501, Loss: 1.066458821296692
Epoch: 0/2, Batch: 50/501, Loss: 0.5157679319381714
Epoch: 0/2, Batch: 100/501, Loss: 0.12106795608997345
Epoch: 0/2, Batch: 150/501, Loss: 0.057508982717990875
Epoch: 0/2, Batch: 200/501, Loss: 0.05051800608634949
Epoch: 0/2, Batch: 250/501, Loss: 0.040825698524713516
Epoch: 0/2, Batch: 300/501, Loss: 0.05167870223522186
Epoch: 0/2, Batch: 350/501, Loss: 0.03849620372056961
Epoch: 0/2, Batch: 400/501, Loss: 0.026828965172171593
Epoch: 0/2, Batch: 450/501, Loss: 0.021661341190338135
Epoch: 0/2, Batch: 500/501, Loss: 0.017453383654356003
Epoch: 1/2, Batch: 0/501, Loss: 0.018749820068478584
Epoch: 1/2, Batch: 50/501, Loss: 0.02531721442937851
Epoch: 1/2, Batch: 100/501, Loss: 0.021940916776657104
Epoch: 1/2, Batch: 150/501, Loss: 0.021481355652213097
Epoch: 1/2, Batch: 200/501, Loss: 0.013598578982055187
Epoch: 1/2, Batch: 250/501, Loss: 0.0321226641535759
Epoch: 1/2, Batch: 300/501, Loss: 0.03448912873864174
Epoch: 1/2, Batch: 3

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 4/10
Epoch: 0/2, Batch: 0/501, Loss: 1.1185169219970703
Epoch: 0/2, Batch: 50/501, Loss: 0.45900189876556396
Epoch: 0/2, Batch: 100/501, Loss: 0.13250985741615295
Epoch: 0/2, Batch: 150/501, Loss: 0.07833289355039597
Epoch: 0/2, Batch: 200/501, Loss: 0.04811972752213478
Epoch: 0/2, Batch: 250/501, Loss: 0.042225755751132965
Epoch: 0/2, Batch: 300/501, Loss: 0.02841750718653202
Epoch: 0/2, Batch: 350/501, Loss: 0.03172757104039192
Epoch: 0/2, Batch: 400/501, Loss: 0.02371467649936676
Epoch: 0/2, Batch: 450/501, Loss: 0.02392943762242794
Epoch: 0/2, Batch: 500/501, Loss: 0.022861652076244354
Epoch: 1/2, Batch: 0/501, Loss: 0.03690163791179657
Epoch: 1/2, Batch: 50/501, Loss: 0.01071824599057436
Epoch: 1/2, Batch: 100/501, Loss: 0.021401330828666687
Epoch: 1/2, Batch: 150/501, Loss: 0.012666112743318081
Epoch: 1/2, Batch: 200/501, Loss: 0.02982257306575775
Epoch: 1/2, Batch: 250/501, Loss: 0.033920422196388245
Epoch: 1/2, Batch: 300/501, Loss: 0.03127976506948471
Epoch: 1/2, Batch: 35

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
results

,accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro,execution_time
0,0.995398,0.995398,0.558875,0.995398,0.341201,0.995398,0.347412,379.121500
1,0.995062,0.995062,0.626600,0.995062,0.365132,0.995062,0.383893,364.071501
2,0.995027,0.995027,0.331676,0.995027,0.333333,0.995027,0.332502,364.300125
3,0.995109,0.995109,0.683628,0.995109,0.361455,0.995109,0.384275,365.182656
4,0.994637,0.994637,0.577167,0.994637,0.352177,0.994637,0.364741,363.327795
5,0.995027,0.995027,0.598084,0.995027,0.366324,0.995027,0.391179,384.738053
6,0.995043,0.995043,0.565520,0.995043,0.346574,0.995043,0.357144,414.651173
7,0.994988,0.994988,0.551710,0.994988,0.359725,0.994988,0.375842,417.224002
8,0.995230,0.995230,0.588105,0.995230,0.363148,0.995230,0.385116,394.022647
9,0.994738,0.994738,0.443821,0.994738,0.346366,0.994738,0.355895,378.653908


In [20]:
results.to_csv(STATS_OUTPUT)